In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
select 
cast(concat(YEAR,'-',MONTH,'-',DAY) as DATE) as DATE
,cast(concat(YEAR,'-',MONTH,'-',DAY,
' ',SUBSTR(SCHEDULED_DEPARTURE,0,2),':',SUBSTR(SCHEDULED_DEPARTURE,3,4)
) as DATETIME) as SCHEDULED_DEPARTURE
,cast(concat(YEAR,'-',MONTH,'-',DAY,
' ',SUBSTR(DEPARTURE_TIME,0,2),':',SUBSTR(DEPARTURE_TIME,3,4)
) as DATETIME) as DEPARTURE_TIME
-- ,cast(concat(YEAR,'-',MONTH,'-',DAY,
-- ' ',SUBSTR(SCHEDULED_TIME,0,2),':',SUBSTR(DEPARTURE_TIME,3,4)
-- ) as DATETIME) as SCHEDULED_TIME
,*
from 
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.FLIGHTS
limit 100;


In [ ]:
--  query for dup check on primary key / unique key
select 
YEAR,
MONTH,
DAY,
FLIGHT_NUMBER,
AIRLINE,
ORIGIN_AIRPORT,
DESTINATION_AIRPORT,
count(*)
from 
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.FLIGHTS
group by
YEAR,
MONTH,
DAY,
FLIGHT_NUMBER,
AIRLINE,
ORIGIN_AIRPORT,
DESTINATION_AIRPORT
having count(*)>1

In [ ]:
-- Helpful query for partition key / cluster key
select 
YEAR,
MONTH,
count(*)
from 
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.FLIGHTS
group by
YEAR,
MONTH
--having count(*)>1

In [ ]:
---
Create  table PH_DATA.NORTHWOODS.FLIGHTS
cluster by LINEAR(YEAR,MONTH)
(
YEAR NUMBER(38,0) NOT NULL,
	MONTH NUMBER(38,0) NOT NULL,
	DAY NUMBER(38,0) NOT NULL,
	DAY_OF_WEEK NUMBER(38,0),
	AIRLINE VARCHAR(16777216) NOT NULL,
	FLIGHT_NUMBER VARCHAR(16777216) NOT NULL,
	TAIL_NUMBER VARCHAR(16777216),
	ORIGIN_AIRPORT VARCHAR(16777216) NOT NULL,
	DESTINATION_AIRPORT VARCHAR(16777216) NOT NULL,
	SCHEDULED_DEPARTURE VARCHAR(16777216),
	DEPARTURE_TIME VARCHAR(16777216),
	DEPARTURE_DELAY NUMBER(38,0),
	TAXI_OUT NUMBER(38,0),
	WHEELS_OFF VARCHAR(16777216),
	SCHEDULED_TIME NUMBER(38,0),
	ELAPSED_TIME NUMBER(38,0),
	AIR_TIME NUMBER(38,0),
	DISTANCE NUMBER(38,0),
	WHEELS_ON NUMBER(38,0),
	TAXI_IN NUMBER(38,0),
	SCHEDULED_ARRIVAL NUMBER(38,0),
	ARRIVAL_TIME VARCHAR(16777216),
	ARRIVAL_DELAY VARCHAR(16777216),
	DIVERTED NUMBER(38,0),
	CANCELLED NUMBER(38,0),
	CANCELLATION_REASON VARCHAR(16777216),
	AIR_SYSTEM_DELAY NUMBER(38,0),
	SECURITY_DELAY NUMBER(38,0),
	AIRLINE_DELAY NUMBER(38,0),
	LATE_AIRCRAFT_DELAY NUMBER(38,0),
	WEATHER_DELAY NUMBER(38,0),
    CONSTRAINT PK_FLIGHTS_KEY PRIMARY KEY (YEAR,
                                            MONTH,
                                            DAY,
                                            FLIGHT_NUMBER,
                                            AIRLINE,
                                            ORIGIN_AIRPORT,
                                            DESTINATION_AIRPORT)
    , CONSTRAINT FK_FLIGHTS_KEY_AIRLINE FOREIGN KEY (AIRLINE) REFERENCES PH_DATA.NORTHWOODS.AIRLINES(IATA_CODE)
    , CONSTRAINT FK_FLIGHTS_KEY_ORIGIN_AIRPORTS FOREIGN KEY (ORIGIN_AIRPORT) REFERENCES PH_DATA.NORTHWOODS.AIRPORTS(IATA_CODE)
    , CONSTRAINT FK_FLIGHTS_KEY_DESTINATION_AIRPORTS FOREIGN KEY (DESTINATION_AIRPORT) REFERENCES PH_DATA.NORTHWOODS.AIRPORTS(IATA_CODE)
            
);


In [ ]:
select *

From INFORMATION_SCHEMA.COLUMNS As C
limit 20;

In [ ]:
use database PH_DATA;
use schema NORTHWOODS;
Select T.TABLE_NAME,
listagg(
(case when 
DATA_TYPE='FLOAT' then 
concat(  
', \n',
-- C.COLUMN_NAME 
--,'!= 
'try_to_decimal(src.',C.COLUMN_NAME,',38,10 ) '
)
when 
DATA_TYPE='NUMBER' then
concat(
', \n',
--C.COLUMN_NAME 
--,'!= 
'try_to_number(src.',C.COLUMN_NAME,',38,10 ) '
)
else
concat( 
', \n',
--C.COLUMN_NAME 
--,'!= 
'src.',C.COLUMN_NAME,''
)
end
)
)

        From (select * from INFORMATION_SCHEMA.COLUMNS
        order by COLUMN_NAME,ORDINAL_POSITION) As C
        join
        INFORMATION_SCHEMA.TABLES As T
        Where C.TABLE_SCHEMA = T.TABLE_SCHEMA
            And C.TABLE_NAME = T.TABLE_NAME
            group by
            T.TABLE_NAME
        --Order By C.ORDINAL_POSITION


        ;
-- Select TABLE_SCHEMA, TABLE_NAME
--     , 
--         (
--         Select ', ' + C.COLUMN_NAME
--         From INFORMATION_SCHEMA.COLUMNS As C
--         Where C.TABLE_SCHEMA = T.TABLE_SCHEMA
--             And C.TABLE_NAME = T.TABLE_NAME
--         Order By C.ORDINAL_POSITION

--         ) As Columns
-- From INFORMATION_SCHEMA.TABLES As T

In [ ]:
create  TABLE FLIGHTS (
	YEAR VARCHAR(16777216),
	MONTH VARCHAR(16777216),
	DAY VARCHAR(16777216),
	DAY_OF_WEEK VARCHAR(16777216),
	AIRLINE VARCHAR(16777216),
	FLIGHT_NUMBER VARCHAR(16777216),
	TAIL_NUMBER VARCHAR(16777216),
	ORIGIN_AIRPORT VARCHAR(16777216),
	DESTINATION_AIRPORT VARCHAR(16777216),
	SCHEDULED_DEPARTURE VARCHAR(16777216),
	DEPARTURE_TIME VARCHAR(16777216),
	DEPARTURE_DELAY VARCHAR(16777216),
	TAXI_OUT VARCHAR(16777216),
	WHEELS_OFF VARCHAR(16777216),
	SCHEDULED_TIME VARCHAR(16777216),
	ELAPSED_TIME VARCHAR(16777216),
	AIR_TIME VARCHAR(16777216),
	DISTANCE VARCHAR(16777216),
	WHEELS_ON VARCHAR(16777216),
	TAXI_IN VARCHAR(16777216),
	SCHEDULED_ARRIVAL VARCHAR(16777216),
	ARRIVAL_TIME VARCHAR(16777216),
	ARRIVAL_DELAY VARCHAR(16777216),
	DIVERTED VARCHAR(16777216),
	CANCELLED VARCHAR(16777216),
	CANCELLATION_REASON VARCHAR(16777216),
	AIR_SYSTEM_DELAY VARCHAR(16777216),
	SECURITY_DELAY VARCHAR(16777216),
	AIRLINE_DELAY VARCHAR(16777216),
	LATE_AIRCRAFT_DELAY VARCHAR(16777216),
	WEATHER_DELAY VARCHAR(16777216)
;;;kkppp);

In [ ]:
, DIVERTED= try_to_number(src.DIVERTED,38,10 ) 
, DAY_OF_WEEK= try_to_number(src.DAY_OF_WEEK,38,10 ) 
, WEATHER_DELAY= try_to_number(src.WEATHER_DELAY,38,10 ) 
, CANCELLED= try_to_number(src.CANCELLED,38,10 ) 
, DEPARTURE_DELAY= try_to_number(src.DEPARTURE_DELAY,38,10 ) 
, SCHEDULED_ARRIVAL= try_to_number(src.SCHEDULED_ARRIVAL,38,10 ) 
, TAXI_OUT= try_to_number(src.TAXI_OUT,38,10 ) 
, DESTINATION_AIRPORT= src.DESTINATION_AIRPORT 
, SCHEDULED_TIME= try_to_number(src.SCHEDULED_TIME,38,10 ) 
, DISTANCE= try_to_number(src.DISTANCE,38,10 ) 
, TAIL_NUMBER= src.TAIL_NUMBER 
, AIR_TIME= try_to_number(src.AIR_TIME,38,10 ) 
, ARRIVAL_DELAY= src.ARRIVAL_DELAY 
, SCHEDULED_DEPARTURE= src.SCHEDULED_DEPARTURE 
, LATE_AIRCRAFT_DELAY= try_to_number(src.LATE_AIRCRAFT_DELAY,38,10 ) 
, YEAR= try_to_number(src.YEAR,38,10 ) 
, AIR_SYSTEM_DELAY= try_to_number(src.AIR_SYSTEM_DELAY,38,10 ) 
, CANCELLATION_REASON= src.CANCELLATION_REASON 
, ARRIVAL_TIME= src.ARRIVAL_TIME 
, SECURITY_DELAY= try_to_number(src.SECURITY_DELAY,38,10 ) 
, MONTH= try_to_number(src.MONTH,38,10 ) 
, ORIGIN_AIRPORT= src.ORIGIN_AIRPORT 
, DEPARTURE_TIME= src.DEPARTURE_TIME 
, WHEELS_ON= try_to_number(src.WHEELS_ON,38,10 ) 
, ELAPSED_TIME= try_to_number(src.ELAPSED_TIME,38,10 ) 
, AIRLINE= src.AIRLINE 
, WHEELS_OFF= src.WHEELS_OFF 
, AIRLINE_DELAY= try_to_number(src.AIRLINE_DELAY,38,10 ) 
, TAXI_IN= try_to_number(src.TAXI_IN,38,10 ) 
, DAY= try_to_number(src.DAY,38,10 ) 
, FLIGHT_NUMBER= src.FLIGHT_NUMBER 


In [ ]:
, MONTH= src.MONTH
, WHEELS_ON= src.WHEELS_ON
, ORIGIN_AIRPORT= src.ORIGIN_AIRPORT
, WHEELS_OFF= src.WHEELS_OFF
, AIRLINE_DELAY= src.AIRLINE_DELAY
, DEPARTURE_TIME= src.DEPARTURE_TIME
, AIRLINE= src.AIRLINE
, ELAPSED_TIME= src.ELAPSED_TIME
, TAXI_OUT= src.TAXI_OUT
, DESTINATION_AIRPORT= src.DESTINATION_AIRPORT
, DISTANCE= src.DISTANCE
, SCHEDULED_TIME= src.SCHEDULED_TIME
, LATE_AIRCRAFT_DELAY= src.LATE_AIRCRAFT_DELAY
, YEAR= src.YEAR
, ARRIVAL_DELAY= src.ARRIVAL_DELAY
, AIR_SYSTEM_DELAY= src.AIR_SYSTEM_DELAY
, SCHEDULED_DEPARTURE= src.SCHEDULED_DEPARTURE
, TAXI_IN= src.TAXI_IN
, DAY= src.DAY
, FLIGHT_NUMBER= src.FLIGHT_NUMBER
, WEATHER_DELAY= src.WEATHER_DELAY
, DAY_OF_WEEK= src.DAY_OF_WEEK
, SCHEDULED_ARRIVAL= src.SCHEDULED_ARRIVAL
, CANCELLED= src.CANCELLED
, DEPARTURE_DELAY= src.DEPARTURE_DELAY
, DIVERTED= src.DIVERTED
, TAIL_NUMBER= src.TAIL_NUMBER
, AIR_TIME= src.AIR_TIME
, SECURITY_DELAY= src.SECURITY_DELAY
, CANCELLATION_REASON= src.CANCELLATION_REASON
, ARRIVAL_TIME= src.ARRIVAL_TIME

In [ ]:
-- create  TABLE PH_DATA.NORTHWOODS.AIRLINES ( 	
-- AIRLINE VARCHAR(16777216), 	IATA_CODE VARCHAR(16777216) 
-- );

In [ ]:
create or replace TABLE PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRLINES ( 	
AIRLINE VARCHAR(16777216), 	IATA_CODE VARCHAR(16777216) 
;;;);

In [ ]:
create  TABLE PH_DATA.NORTHWOODS.AIRLINES ( 	
AIRLINE VARCHAR(16777216), 	IATA_CODE VARCHAR(16777216) ,
CONSTRAINT PK_AIRLINES_KEY PRIMARY KEY (IATA_CODE)
);

In [ ]:
create  TABLE PH_DATA.NORTHWOODS.AIRPORTS (
	IATA_CODE VARCHAR(16777216),
	AIRPORT VARCHAR(16777216),
	CITY VARCHAR(16777216),
	STATE VARCHAR(16777216),
	COUNTRY VARCHAR(16777216),
	LATITUDE FLOAT,
	LONGITUDE FLOAT,
    CONSTRAINT PK_AIRPORTS_KEY PRIMARY KEY (IATA_CODE)
);

In [ ]:
select
--distinct cast(src.LATITUDE as float) LATITUDE
--,
try_to_decimal(src.LATITUDE,38,37 ),
try_to_number(src.LATITUDE,38,37 ),
LATITUDE,
-- to_char(substr(LATITUDE,0, 39)::number(38, 37),'9.9999999999999999999999999999999999999'),
NVL(LATITUDE,null),
-- to_char(substr(
-- (
-- case when 
-- try_to_decimal(src.LATITUDE,20,10 ) is not null
-- and LATITUDE is not null
-- then 
-- LATITUDE else null end 
-- )
-- ,0, 39)::number(38, 37),'9.9999999999999999999999999999999999999'),
NVL(
cast(
(
case when 
try_to_decimal(src.LATITUDE,20,10 ) is not null
and LATITUDE is not null
then 
NVL(LATITUDE,null) else null end 
)
as
number(38,35)
)
,null)
,
case when LATITUDE=cast(try_to_decimal(src.LATITUDE,20,10 ) as varchar) then
'okay' else 'issue' end
from
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRPORTS src
-- where 
-- try_to_decimal(src.LATITUDE,20,10 ) is null
-- and LATITUDE is not null

In [ ]:
create  TABLE PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRPORTS (
	IATA_CODE VARCHAR(16777216),
	AIRPORT VARCHAR(16777216),
	CITY VARCHAR(16777216),
	STATE VARCHAR(16777216),
	COUNTRY VARCHAR(16777216),
	LATITUDE VARCHAR(16777216),
	-- LONGITUDE VARCHAR(16777216)
);

In [ ]:
select get_ddl('table','PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRPORTS')

In [ ]:
Update PH_DATA.NORTHWOODS.AIRLINES
set AIRLINE=src.AIRLINE
from 
PH_DATA.NORTHWOODS.AIRLINES tgt
right join
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRLINES src
on src.IATA_CODE=tgt.IATA_CODE
where
src.AIRLINE!=tgt.AIRLINE
;

Insert into PH_DATA.NORTHWOODS.AIRLINES
(
IATA_CODE,
AIRLINE
)

select 
src.IATA_CODE,
src.AIRLINE
from
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRLINES src
where src.IATA_CODE not in (
select tgt.IATA_CODE
from
PH_DATA.NORTHWOODS.AIRLINES tgt
)

;
select *
from
PH_DATA.NORTHWOODS.AIRLINES

In [ ]:
Update PH_DATA.NORTHWOODS.AIRPORTS
set 
AIRPORT=src.AIRPORT,
CITY =src.CITY,
STATE =src.STATE,
COUNTRY =src.COUNTRY,
LATITUDE =try_to_decimal(src.LATITUDE,38,10 ),
LONGITUDE =try_to_decimal(src.LONGITUDE,38,10 )
from 
PH_DATA.NORTHWOODS.AIRPORTS tgt
right join
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRPORTS src
on src.IATA_CODE=tgt.IATA_CODE
where
(src.AIRPORT!=tgt.AIRPORT
or 
src.CITY!=tgt.CITY
or
src.STATE!=tgt.STATE
or
src.COUNTRY!=tgt.COUNTRY
or
src.AIRPORT!=tgt.AIRPORT
or
try_to_decimal(src.LATITUDE,38,10 )!=tgt.LATITUDE
or
try_to_decimal(src.LONGITUDE,38,10 )!=tgt.LONGITUDE
)
;

Insert into PH_DATA.NORTHWOODS.AIRPORTS
(
IATA_CODE,
AIRPORT,
CITY ,
STATE,
COUNTRY ,
LATITUDE ,
LONGITUDE 
)
select 
src.IATA_CODE,
src.AIRPORT,
src.CITY ,
src.STATE,
src.COUNTRY ,
try_to_decimal(src.LATITUDE,38,10 ) LATITUDE,
try_to_decimal(src.LONGITUDE,38,10 ) LONGITUDE
from
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRPORTS src
where src.IATA_CODE not in (
select tgt.IATA_CODE
from
PH_DATA.NORTHWOODS.AIRPORTS tgt
)

;

In [ ]:
Update PH_DATA.NORTHWOODS.FLIGHTS
set 
  DIVERTED= try_to_number(src.DIVERTED,38,10 ) 
, DAY_OF_WEEK= try_to_number(src.DAY_OF_WEEK,38,10 ) 
, WEATHER_DELAY= try_to_number(src.WEATHER_DELAY,38,10 ) 
, CANCELLED= try_to_number(src.CANCELLED,38,10 ) 
, DEPARTURE_DELAY= try_to_number(src.DEPARTURE_DELAY,38,10 ) 
, SCHEDULED_ARRIVAL= try_to_number(src.SCHEDULED_ARRIVAL,38,10 ) 
, TAXI_OUT= try_to_number(src.TAXI_OUT,38,10 ) 
, DESTINATION_AIRPORT= src.DESTINATION_AIRPORT 
, SCHEDULED_TIME= try_to_number(src.SCHEDULED_TIME,38,10 ) 
, DISTANCE= try_to_number(src.DISTANCE,38,10 ) 
, TAIL_NUMBER= src.TAIL_NUMBER 
, AIR_TIME= try_to_number(src.AIR_TIME,38,10 ) 
, ARRIVAL_DELAY= src.ARRIVAL_DELAY 
, SCHEDULED_DEPARTURE= src.SCHEDULED_DEPARTURE 
, LATE_AIRCRAFT_DELAY= try_to_number(src.LATE_AIRCRAFT_DELAY,38,10 ) 
, YEAR= try_to_number(src.YEAR,38,10 ) 
, AIR_SYSTEM_DELAY= try_to_number(src.AIR_SYSTEM_DELAY,38,10 ) 
, CANCELLATION_REASON= src.CANCELLATION_REASON 
, ARRIVAL_TIME= src.ARRIVAL_TIME 
, SECURITY_DELAY= try_to_number(src.SECURITY_DELAY,38,10 ) 
, MONTH= try_to_number(src.MONTH,38,10 ) 
, ORIGIN_AIRPORT= src.ORIGIN_AIRPORT 
, DEPARTURE_TIME= src.DEPARTURE_TIME 
, WHEELS_ON= try_to_number(src.WHEELS_ON,38,10 ) 
, ELAPSED_TIME= try_to_number(src.ELAPSED_TIME,38,10 ) 
, AIRLINE= src.AIRLINE 
, WHEELS_OFF= src.WHEELS_OFF 
, AIRLINE_DELAY= try_to_number(src.AIRLINE_DELAY,38,10 ) 
, TAXI_IN= try_to_number(src.TAXI_IN,38,10 ) 
, DAY= try_to_number(src.DAY,38,10 ) 
, FLIGHT_NUMBER= src.FLIGHT_NUMBER 
from 
PH_DATA.NORTHWOODS.FLIGHTS tgt
right join
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.FLIGHTS src
on src.YEAR=tgt.YEAR
and src.MONTH=tgt.MONTH
and src.DAY=tgt.DAY
and src.FLIGHT_NUMBER=tgt.FLIGHT_NUMBER
and src.AIRLINE=tgt.AIRLINE
and src.ORIGIN_AIRPORT=tgt.ORIGIN_AIRPORT
and src.DESTINATION_AIRPORT=tgt.DESTINATION_AIRPORT
where
(
tgt.AIR_TIME!= try_to_number(src.AIR_TIME,38,10 ) or 
tgt.TAIL_NUMBER!= src.TAIL_NUMBER or
tgt.LATE_AIRCRAFT_DELAY!= try_to_number(src.LATE_AIRCRAFT_DELAY,38,10 ) or 
tgt.SCHEDULED_DEPARTURE!= src.SCHEDULED_DEPARTURE or
tgt.ARRIVAL_DELAY!= src.ARRIVAL_DELAY or
tgt.AIR_SYSTEM_DELAY!= try_to_number(src.AIR_SYSTEM_DELAY,38,10 ) or 
tgt.SCHEDULED_ARRIVAL!= try_to_number(src.SCHEDULED_ARRIVAL,38,10 ) or 
tgt.WEATHER_DELAY!= try_to_number(src.WEATHER_DELAY,38,10 ) or 
tgt.DAY_OF_WEEK!= try_to_number(src.DAY_OF_WEEK,38,10 ) or 
tgt.CANCELLED!= try_to_number(src.CANCELLED,38,10 ) or 
tgt.DEPARTURE_DELAY!= try_to_number(src.DEPARTURE_DELAY,38,10 ) or 
tgt.DISTANCE!= try_to_number(src.DISTANCE,38,10 ) or 
tgt.TAXI_OUT!= try_to_number(src.TAXI_OUT,38,10 ) or 
tgt.SCHEDULED_TIME!= try_to_number(src.SCHEDULED_TIME,38,10 ) or 
tgt.WHEELS_ON!= try_to_number(src.WHEELS_ON,38,10 ) or 
tgt.DEPARTURE_TIME!= src.DEPARTURE_TIME or
tgt.WHEELS_OFF!= src.WHEELS_OFF or
tgt.AIRLINE_DELAY!= try_to_number(src.AIRLINE_DELAY,38,10 ) or 
tgt.ELAPSED_TIME!= try_to_number(src.ELAPSED_TIME,38,10 ) or 
tgt.TAXI_IN!= try_to_number(src.TAXI_IN,38,10 ) or 
tgt.DIVERTED!= try_to_number(src.DIVERTED,38,10 ) or 
tgt.SECURITY_DELAY!= try_to_number(src.SECURITY_DELAY,38,10 ) or 
tgt.CANCELLATION_REASON!= src.CANCELLATION_REASON or
tgt.ARRIVAL_TIME!= src.ARRIVAL_TIME 

)
;

Insert into PH_DATA.NORTHWOODS.FLIGHTS
(
AIRLINE, 
AIRLINE_DELAY, 
AIR_SYSTEM_DELAY, 
AIR_TIME, 
ARRIVAL_DELAY, 
ARRIVAL_TIME, 
CANCELLATION_REASON, 
CANCELLED, 
DAY, 
DAY_OF_WEEK, 
DEPARTURE_DELAY, 
DEPARTURE_TIME, 
DESTINATION_AIRPORT, 
DISTANCE, 
DIVERTED, 
ELAPSED_TIME, 
FLIGHT_NUMBER, 
ORIGIN_AIRPORT, 
LATE_AIRCRAFT_DELAY, 
MONTH, 
SCHEDULED_ARRIVAL, 
SCHEDULED_DEPARTURE, 
SCHEDULED_TIME, 
SECURITY_DELAY, 
TAIL_NUMBER, 
TAXI_IN, 
TAXI_OUT, 
WEATHER_DELAY, 
WHEELS_OFF, 
WHEELS_ON, 
YEAR
)
select 

src.AIRLINE, 
try_to_number(src.AIRLINE_DELAY,38,10 ) , 
try_to_number(src.AIR_SYSTEM_DELAY,38,10 ) , 
try_to_number(src.AIR_TIME,38,10 ) , 
src.ARRIVAL_DELAY, 
src.ARRIVAL_TIME, 

src.CANCELLATION_REASON, 
try_to_number(src.CANCELLED,38,10 ) ,

try_to_number(src.DAY,38,10 ) , 
try_to_number(src.DAY_OF_WEEK,38,10 ) , 
try_to_number(src.DEPARTURE_DELAY,38,10 ) , 
src.DEPARTURE_TIME, 
src.DESTINATION_AIRPORT, 
try_to_number(src.DISTANCE,38,10 ) , 
try_to_number(src.DIVERTED,38,10 ) , 

try_to_number(src.ELAPSED_TIME,38,10 ) , 
src.FLIGHT_NUMBER, 
src.ORIGIN_AIRPORT,

try_to_number(src.LATE_AIRCRAFT_DELAY,38,10 ) , 
try_to_number(src.MONTH,38,10 ) ,

try_to_number(src.SCHEDULED_ARRIVAL,38,10 ) , 
src.SCHEDULED_DEPARTURE, 
try_to_number(src.SCHEDULED_TIME,38,10 ) , 
try_to_number(src.SECURITY_DELAY,38,10 ) , 
src.TAIL_NUMBER, 
try_to_number(src.TAXI_IN,38,10 ) , 
try_to_number(src.TAXI_OUT,38,10 ) , 
try_to_number(src.WEATHER_DELAY,38,10 ) , 
src.WHEELS_OFF, 
try_to_number(src.WHEELS_ON,38,10 ) , 
try_to_number(src.YEAR,38,10 ) 

 

from
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.FLIGHTS src
-- on src.YEAR=tgt.YEAR
-- and src.MONTH=tgt.MONTH
-- and src.DAY=tgt.DAY
-- and src.FLIGHT_NUMBER=tgt.FLIGHT_NUMBER
-- and src.AIRLINE=tgt.AIRLINE
-- and src.ORIGIN_AIRPORT=tgt.ORIGIN_AIRPORT
-- and src.DESTINATION_AIRPORT=tgt.DESTINATION_AIRPORT

where concat(src.YEAR,src.MONTH,src.DAY,src.FLIGHT_NUMBER,src.AIRLINE,src.ORIGIN_AIRPORT,src.DESTINATION_AIRPORT) not in (
select concat(tgt.YEAR,tgt.MONTH,tgt.DAY,tgt.FLIGHT_NUMBER,tgt.AIRLINE,tgt.ORIGIN_AIRPORT,tgt.DESTINATION_AIRPORT)
from
PH_DATA.NORTHWOODS.FLIGHTS tgt
)

;

In [ ]:
select *
from 
PH_DATA.NORTHWOODS.AIRLINES tgt
left join
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRLINES src
on tgt.IATA_CODE=src.IATA_CODE

In [ ]:
Insert into PH_DATA.NORTHWOODS.AIRLINES
(
IATA_CODE,
AIRLINE
)
as
(
select 
IATA_CODE,
AIRLINE
from
PH_DATA.NORTHWOODS.AIRLINES tgt
left join
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRLINES src
on src.IATA_CODE!=tgt.IATA_CODE
)
;

In [ ]:
(
select 
YEAR,
MONTH,
AIRLINE,
ORIGIN_AIRPORT,
count (distinct (FLIGHT_NUMBER))
from 
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.FLIGHTS
group by
AIRLINE,
ORIGIN_AIRPORT,
YEAR,
MONTH
order by 
YEAR,
MONTH,
AIRLINE,
ORIGIN_AIRPORT
) monthly
left join
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRLINES a
on monthly.AIRLINE=a.IATA_CODE
left join
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRPORTS a
on monthly.AIRLINE=a.IATA_CODE


In [ ]:
select *
from 
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRLINES
limit 100;

In [ ]:
select *
from 
PH_DATA_RAW.NORTHWOODS_RAW_LAYER.AIRPORTS
limit 100;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")